In [1]:
import numpy as np
import pathlib
from nilearn.decomposition import CanICA
def ICA_decomposition(func_filenames, group, i):
    fast_ica = CanICA(n_components=30,
                    memory="nilearn_cache", memory_level=2,
                    mask_strategy='whole-brain-template',
                    do_cca=False,
                    random_state=0)
    fast_ica.fit(func_filenames)
    P = pathlib.Path('/data/origami/niusha/code/local-experiment/io/ICAs')
    ICA_s = fast_ica.components_img_
    ICA_s.to_filename(P / (f'ICAs_{group}_{i}.nii.gz'))
    return ICA_s

In [2]:
from nilearn import masking
from nilearn.image import iter_img
from nilearn.masking import apply_mask

def Means_after_masking(ICAs,DBM_maps):

    means_after_mask = np.zeros((30,DBM_maps.shape[3]))
    for i, cur_img in enumerate(iter_img(ICAs)):
        mask_img = masking.compute_epi_mask(cur_img)
        means_after_mask[i,:] = np.mean(apply_mask(DBM_maps,mask_img),axis=1)
        
    return means_after_mask

In [3]:
import pandas as pd

subject_df = pd.read_csv("/data/origami/niusha/input/subject_IDs.csv")

N = len(subject_df.ID)
ID_map = dict(zip(range(N),subject_df.ID))

Healthy_index = np.where(subject_df.PD == 0)
Healthy_subject = subject_df.ID.iloc[Healthy_index]

PD_index = np.where(subject_df.PD == 1)
PD_subject = subject_df.ID.iloc[PD_index]

In [4]:
from nilearn.image import load_img
Original_DBMs = load_img("/data/origami/niusha/out/DBM_data.nii")

In [5]:
from random import sample
from nilearn.image import index_img
from scipy.stats import ttest_ind

P_vals = np.zeros((10,30))
T_vals = np.zeros((10,30))

for j in range(10):

    PD_sample = sample((PD_index[0]).tolist(), len(PD_index[0]))
    Healthy_sample = sample((Healthy_index[0]).tolist(), len(Healthy_index[0]))

    PD_bootstrapped_cohort = index_img(Original_DBMs, PD_sample)
    Healthy_bootstrapped_cohort = index_img(Original_DBMs, Healthy_sample)

    PD_ICAs = ICA_decomposition(PD_bootstrapped_cohort, "PD", j)
    Healthy_ICAs = ICA_decomposition(Healthy_bootstrapped_cohort, "Healthy", j)

    PD_means = Means_after_masking(PD_ICAs,PD_bootstrapped_cohort)
    Healthy_means = Means_after_masking(Healthy_ICAs,Healthy_bootstrapped_cohort)

    for i in range(30):
        T_vals[j,i], P_vals[j,i] = ttest_ind(PD_means[:,i], Healthy_means[:,i])

/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/joblib/memory.py:632: UserWarning: Cannot inspect object functools.partial(<function compute_multi_brain_mask at 0x7ff6b3795b20>, mask_type='whole-brain'), ignore list will not work.
  return hashing.hash(filter_args(self.func, self.ignore, args, kwargs),
/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/nilearn/maskers/multi_nifti_masker.py:258: JobLibCollisionWarning: Cannot detect name collisions for function 'unknown'
  self.mask_img_ = self._cache(
/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/nilearn/masking.py:568: UserWarning: Persisting input arguments took 3.47s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they c

In [7]:
# P_vals_reshaped = P_vals.reshape(P_vals.shape[0], -1)
np.savetxt("/data/origami/niusha/code/local-experiment/io/p_values.txt", P_vals)

# T_vals_reshaped = T_vals.reshape(T_vals.shape[0], -1)
np.savetxt("/data/origami/niusha/code/local-experiment/io/t_stat.txt", T_vals)

In [8]:
P_vals

array([[8.38236637e-10, 5.36940622e-32, 3.98633303e-28,            nan,
        1.64428193e-41, 3.82397658e-59, 6.61387252e-33, 4.35098975e-29,
        5.34956358e-39, 1.38908876e-40, 1.60839490e-40, 5.93274919e-28,
        5.44435909e-02, 6.34770551e-52, 7.63137514e-15, 5.23911931e-48,
        6.00503488e-05, 4.64837781e-29, 4.40831837e-45, 7.63326161e-52,
        5.45999644e-05, 2.33350917e-33, 6.80626946e-06, 5.00711081e-16,
        1.60709683e-44, 6.72982107e-09, 5.22616107e-61, 1.84562314e-02,
        1.99421056e-33, 3.05875632e-04],
       [1.70240140e-40, 2.24403341e-32, 3.45483822e-36, 5.31481945e-39,
        3.42947525e-47, 2.39159537e-52, 9.22191263e-01, 1.05179073e-18,
        1.55665932e-04, 2.86429528e-32,            nan, 7.71721777e-45,
        4.47054279e-55, 1.29046801e-01, 1.67739465e-28, 1.21629640e-21,
        1.14063475e-49, 1.56943198e-15, 3.10346012e-42, 1.07092371e-55,
        2.55415405e-53, 2.00808605e-38, 2.88785406e-34, 1.13476033e-31,
        4.15201779e-52,

In [ ]:
P_vals_original = np.zeros((1,30))
T_vals_original = np.zeros((1,30))
PD_cohort = index_img(Original_DBMs, (PD_index[0]).tolist())
Healthy_cohort = index_img(Original_DBMs, (Healthy_index[0]).tolist())

PD_ICAs = ICA_decomposition(PD_cohort, "PD", "")
Healthy_ICAs = ICA_decomposition(Healthy_cohort, "Healthy", "")

PD_means = Means_after_masking(PD_ICAs,PD_cohort)
Healthy_means = Means_after_masking(Healthy_ICAs,Healthy_cohort)

for i in range(30):
    T_vals_original[0,i], P_vals_original[0,i] = ttest_ind(PD_means[:,i], Healthy_means[:,i])

In [11]:
np.savetxt("/data/origami/niusha/code/local-experiment/io/p_values_original.txt", P_vals_original)
np.savetxt("/data/origami/niusha/code/local-experiment/io/t_stat_original.txt", T_vals_original)

In [14]:
P_vals_whole = np.zeros((10,30))
T_vals_whole = np.zeros((10,30))

for j in range(10):

    PD_sample = sample((PD_index[0]).tolist(), len(PD_index[0]))
    Healthy_sample = sample((Healthy_index[0]).tolist(), len(Healthy_index[0]))

    PD_bootstrapped_cohort = index_img(Original_DBMs, PD_sample)
    Healthy_bootstrapped_cohort = index_img(Original_DBMs, Healthy_sample)
    Whole_bootstrapped_cohort = index_img(Original_DBMs, PD_sample + Healthy_sample)

    whole_ICAs = ICA_decomposition(Whole_bootstrapped_cohort, "whole", j)

    PD_means = Means_after_masking(whole_ICAs,PD_bootstrapped_cohort)
    Healthy_means = Means_after_masking(whole_ICAs,Healthy_bootstrapped_cohort)

    for i in range(30):
        T_vals_whole[j,i], P_vals_whole[j,i] = ttest_ind(PD_means[:,i], Healthy_means[:,i])

/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/joblib/memory.py:632: UserWarning: Cannot inspect object functools.partial(<function compute_multi_brain_mask at 0x7ff6b3795b20>, mask_type='whole-brain'), ignore list will not work.
  return hashing.hash(filter_args(self.func, self.ignore, args, kwargs),
/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/nilearn/masking.py:568: UserWarning: Persisting input arguments took 5.15s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  resampled_template = cache(resampling.resample_to_img, memory)(
/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/joblib/memory.py:810: UserWarning: Cannot inspect object functools

In [15]:
np.savetxt("/data/origami/niusha/code/local-experiment/io/p_values_whole.txt", P_vals_whole)
np.savetxt("/data/origami/niusha/code/local-experiment/io/t_stat_whole.txt", T_vals_whole)

In [16]:
P_vals_whole_original = np.zeros((1,30))
T_vals_whole_original = np.zeros((1,30))


PD_sample = (PD_index[0]).tolist()
Healthy_sample = (Healthy_index[0]).tolist()

PD_bootstrapped_cohort = index_img(Original_DBMs, PD_sample)
Healthy_bootstrapped_cohort = index_img(Original_DBMs, Healthy_sample)
Whole_bootstrapped_cohort = index_img(Original_DBMs, PD_sample + Healthy_sample)

whole_ICAs = ICA_decomposition(Whole_bootstrapped_cohort, "whole", "original")

PD_means = Means_after_masking(whole_ICAs,PD_bootstrapped_cohort)
Healthy_means = Means_after_masking(whole_ICAs,Healthy_bootstrapped_cohort)

for i in range(30):
    T_vals_whole_original[0,i], P_vals_whole_original[0,i] = ttest_ind(PD_means[:,i], Healthy_means[:,i])

/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/joblib/memory.py:632: UserWarning: Cannot inspect object functools.partial(<function compute_multi_brain_mask at 0x7ff6b3795b20>, mask_type='whole-brain'), ignore list will not work.
  return hashing.hash(filter_args(self.func, self.ignore, args, kwargs),
/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/nilearn/masking.py:568: UserWarning: Persisting input arguments took 5.16s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  resampled_template = cache(resampling.resample_to_img, memory)(
/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/joblib/memory.py:810: UserWarning: Cannot inspect object functools

In [17]:
np.savetxt("/data/origami/niusha/code/local-experiment/io/p_values_whole_original.txt", P_vals_whole_original)
np.savetxt("/data/origami/niusha/code/local-experiment/io/t_stat_whole_original.txt", T_vals_whole_original)

In [ ]:
# from nilearn.plotting import plot_prob_atlas

# # Plot all ICA components together
# plot_prob_atlas(canica_components_img, title='All ICA components')

# from nilearn.image import iter_img
# from nilearn.plotting import plot_stat_map, show

# for i, cur_img in enumerate(iter_img(canica_components_img)):
#     plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
#                   cut_coords=1, colorbar=False)

In [ ]:
# from nilearn.glm import threshold_stats_img

# ICA_tr = threshold_stats_img(stat_img=ICAs, threshold=3.0)
#The output is not compatible with mask and also I dont know what is the ouput of ICA. I tought it would be z stat, but when I set thr=3 I receive warning regarding the amount